<small><i>This notebook was put together by [Wesley Beckner](http://wesleybeckner.github.io/).</i></small>

In [33]:
import json
import os
import sys
import pandas as pd
import numpy as np
from __future__ import print_function

#rdkit modules
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors.MoleculeDescriptors import MolecularDescriptorCalculator as Calculator


#scikit modules
from sklearn.linear_model import Lasso
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler

#plot modules and formatting
import matplotlib.pyplot as plt
%matplotlib inline
plt.rc("figure", facecolor="white")
params = {
    'lines.markersize' : 10,
    'axes.labelsize': 20,
    'font.size': 20,
    'legend.fontsize': 20,
    'xtick.labelsize': 20,
    'ytick.labelsize': 20,
    'text.usetex': False,
   }
tableau20 = [(31, 119, 180), (174, 199, 232), (255, 127, 14), (255, 187, 120),    
             (44, 160, 44), (152, 223, 138), (214, 39, 40), (255, 152, 150),    
             (148, 103, 189), (197, 176, 213), (140, 86, 75), (196, 156, 148),    
             (227, 119, 194), (247, 182, 210), (127, 127, 127), (199, 199, 199),    
             (188, 189, 34), (219, 219, 141), (23, 190, 207), (158, 218, 229)]   
# Scale the RGB values to the [0, 1] range, which is the format matplotlib accepts.    
for i in range(len(tableau20)):    
    r, g, b = tableau20[i]    
    tableau20[i] = (r / 255., g / 255., b / 255.) 

#grab our checkName code
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import salty

In [4]:
outer_old = pd.DataFrame()
outer_new = pd.DataFrame()

for i in range(2266):
    with open("../salty/data/DENSITY/%s.json" % str(i+1)) as json_file:
        
        #grab data, data headers (names), the salt name
        json_full = json.load(json_file)
        json_data = pd.DataFrame(json_full['data'])
        json_datanames = np.array(json_full['dhead'])
        json_data.columns =  json_datanames
        json_saltname = pd.DataFrame(json_full['components'])
        print(json_saltname.iloc[0][3])
        
        inner_old = pd.DataFrame()
        inner_new = pd.DataFrame()
        
        #loop through the columns of the data, note that some of the 
        #json files are missing pressure data. 
        for indexer in range(len(json_data.columns)):
            grab=json_data.columns[indexer]
            list = json_data[grab]
            my_list = [l[0] for l in list]
            dfmy_list = pd.DataFrame(my_list)
            dfmy_list.columns = [json_datanames[indexer][0]]
            inner_new = pd.concat([dfmy_list, inner_old], axis=1)
            inner_old = inner_new
            
        #add the name of the salt    
        inner_old['salt_name']=json_saltname.iloc[0][3]           
        
        #add to the growing dataframe
        outer_new = pd.concat([inner_old, outer_old], axis=0)
        outer_old = outer_new
print(outer_old)
pd.DataFrame.to_csv(outer_old, path_or_buf='../salty/data/density.csv', index=False)

1-hexyl-3-methylimidazolium bis[(trifluoromethyl)sulfonyl]imide
1-butyl-3-methylimidazolium tetrafluoroborate
1-butyl-3-methylimidazolium methylsulfate
1-ethyl-3-methylimidazolium ethyl sulfate
2-hydroxy-N-methylethanaminium pentanoate
2-hydroxy-N-methylethanaminium propionate
N-methyl-2-hydroxyethylammonium acetate
2-hydroxy-N-methylethanaminium formate
2-hydroxy-N-methylethanaminium isobutyrate
N-methyl-2-hydroxyethylammonium butanoate
1-butyl-3-methylimidazolium bis(trifluoromethylsulfonyl)imide
1-ethyl-3-methylimidazolium ethyl sulfate
1-butyl-3-methylimidazolium tetrafluoroborate
1-butyl-3-methylimidazolium hexafluorophosphate
1-hexyl-3-methylimidazolium hexafluorophosphate
1-ethyl-3-methylimidazolium tetrafluoroborate
1-octyl-3-methylimidazolium hexafluorophosphate
1-hexyl-3-methylimidazolium tetrafluoroborate
tetradecyl(trihexyl)phosphonium dicyanamide
1-butyl-3-methylimidazolium bis(trifluoromethylsulfonyl)imide
1-butyl-3-methylimidazolium octyl sulfate
1-butyl-3-methylimidazol

In [ ]:
###run salts through checkName and add SMILES
salts=pd.read_csv('../salty/data/density.csv',delimiter=',')
salts['salt_SMILES'] = np.nan
for a, b in enumerate(salts['salt_name']):        
    try:
        salts['salt_SMILES'][a] = salty.checkName(b)
    except: 
        pass 

database found
query not found
database found
query not found
database found
user has queried with a name
your query has returned CCN1C=C[N+](=C1)C,C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F
database found
user has queried with a name
your query has returned CCN1C=C[N+](=C1)C,C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F
database found
query not found
database found
query not found
database found
query not found
database found
query not found
database found
query not found
database found
query not found
database found
query not found
database found
query not found
database found
query not found
database found
query not found
database found
query not found
database found
query not found
database found
query not found
database found
query not found
database found
query not found
database found
query not found
database found
query not found
database found
query not found
database found
user has queried with a name
your query has returned CCN1C=C[N+](=C1)C,[B-](F)(F)(F)F
database found
user has q

In [20]:
###remove all the salts not found in our database and
###save the truncated salt data
salts=pd.read_csv('../salty/data/salts_with_smiles.csv',delimiter=',')
salts = salts.loc[salts['salt_SMILES'] != "0"]
salts = salts.loc[salts['salt_SMILES'] != "0.0"]
pd.DataFrame.to_csv(salts, path_or_buf='../salty/data/salts_with_smiles.csv', index=False)

In [21]:
###read in the salts csv; add cat_name, an_name, cat_SMILES, and an_SMILES
###columns. Parse salt_name into cation and anion components and append to
###respective columns. Make the call to checkName with those entries and
###append to respective SMILES columns.
salts=pd.read_csv('../salty/data/salts_with_smiles.csv',delimiter=',')
salts['cation_name'] = np.nan
salts['anion_name'] = np.nan
salts['cation_SMILES'] = np.nan
salts['anion_SMILES'] = np.nan
for i in range(salts.shape[0]):
    salts.ix[i, 'cation_name'] = salts['salt_name'].iloc[i].split()[0]
    salts.ix[i, 'cation_SMILES'] = salty.checkName(salts['cation_name'].iloc[i])
    
    if len(salts['salt_name'].iloc[i].split()) == 2:
        salts.ix[i, 'anion_name'] = salts['salt_name'].iloc[i].split()[1]
    else:
        print("long anion name")
        salts.ix[i, 'anion_name'] = (salts['salt_name'].iloc[i].split()[1] + \
        " " + salts['salt_name'].iloc[i].split()[2])
    salts.ix[i, 'anion_SMILES'] = salty.checkName(salts['anion_name'].iloc[i])
salts


database found
user has queried with a name
your query has returned CCN1C=C[N+](=C1)C
database found
user has queried with a name
your query has returned C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F
database found
user has queried with a name
your query has returned CCN1C=C[N+](=C1)C
database found
user has queried with a name
your query has returned [B-](F)(F)(F)F
database found
user has queried with a name
your query has returned CCN1C=C[N+](=C1)C
database found
user has queried with a name
your query has returned [B-](F)(F)(F)F
database found
user has queried with a name
your query has returned CCN1C=C[N+](=C1)C
long anion name
database found
user has queried with a name
your query has returned CCOS(=O)(=O)[O-]

database found
user has queried with a name
your query has returned CCN1C=C[N+](=C1)C
database found
user has queried with a name
your query has returned C(F)(F)(F)S(=O)(=O)[O-]
database found
user has queried with a name
your query has returned CCN1C=C[N+](=C1)C
database found


user has queried with a name
your query has returned CCOS(=O)(=O)[O-]

database found
user has queried with a name
your query has returned CCN1C=C[N+](=C1)C
long anion name
database found
user has queried with a name
your query has returned CCOS(=O)(=O)[O-]

database found
user has queried with a name
your query has returned CCN1C=C[N+](=C1)C
long anion name
database found
user has queried with a name
your query has returned CCOS(=O)(=O)[O-]

database found
user has queried with a name
your query has returned CCN1C=C[N+](=C1)C
database found
user has queried with a name
your query has returned [Cl-]
database found
user has queried with a name
your query has returned CCN1C=C[N+](=C1)C
long anion name
database found
user has queried with a name
your query has returned CCOS(=O)(=O)[O-]

database found
user has queried with a name
your query has returned CCN1C=C[N+](=C1)C
long anion name
database found
user has queried with a name
your query has returned CCOS(=O)(=O)[O-]

database found
us

user has queried with a name
your query has returned C(C(F)(F)S(=O)(=O)NS(=O)(=O)C(C(F)(F)F)(F)F)(F)(F)F
database found
user has queried with a name
your query has returned CCN1C=C[N+](=C1)C
database found
user has queried with a name
your query has returned C(C(F)(F)S(=O)(=O)NS(=O)(=O)C(C(F)(F)F)(F)F)(F)(F)F
database found
user has queried with a name
your query has returned CCN1C=C[N+](=C1)C
database found
user has queried with a name
your query has returned C(C(F)(F)S(=O)(=O)NS(=O)(=O)C(C(F)(F)F)(F)F)(F)(F)F
database found
user has queried with a name
your query has returned CCN1C=C[N+](=C1)C
database found
user has queried with a name
your query has returned C(C(F)(F)S(=O)(=O)NS(=O)(=O)C(C(F)(F)F)(F)F)(F)(F)F
database found
user has queried with a name
your query has returned CCN1C=C[N+](=C1)C
database found
user has queried with a name
your query has returned C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F
database found
user has queried with a name
your query has returned CCN1C=C[N+](=C

user has queried with a name
your query has returned [B-](F)(F)(F)F
database found
user has queried with a name
your query has returned CCN1C=C[N+](=C1)C
database found
user has queried with a name
your query has returned [B-](F)(F)(F)F
database found
user has queried with a name
your query has returned CCN1C=C[N+](=C1)C
database found
user has queried with a name
your query has returned [B-](F)(F)(F)F
database found
user has queried with a name
your query has returned CCN1C=C[N+](=C1)C
database found
user has queried with a name
your query has returned C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F
database found
user has queried with a name
your query has returned CCN1C=C[N+](=C1)C
database found
user has queried with a name
your query has returned C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F
database found
user has queried with a name
your query has returned CCN1C=C[N+](=C1)C
database found
user has queried with a name
your query has returned C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F
database 

user has queried with a name
your query has returned CCN1C=C[N+](=C1)C
database found
user has queried with a name
your query has returned C(F)(F)(F)S(=O)(=O)[O-]
database found
user has queried with a name
your query has returned CCN1C=C[N+](=C1)C
database found
user has queried with a name
your query has returned C(F)(F)(F)S(=O)(=O)[O-]
database found
user has queried with a name
your query has returned CCN1C=C[N+](=C1)C
database found
user has queried with a name
your query has returned C(F)(F)(F)S(=O)(=O)[O-]
database found
user has queried with a name
your query has returned CCN1C=C[N+](=C1)C
database found
user has queried with a name
your query has returned [B-](F)(F)(F)F
database found
user has queried with a name
your query has returned CCN1C=C[N+](=C1)C
database found
user has queried with a name
your query has returned [B-](F)(F)(F)F
database found
user has queried with a name
your query has returned CCN1C=C[N+](=C1)C
database found
user has queried with a name
your query ha

...

user has queried with a name
your query has returned CCOS(=O)(=O)[O-]

database found
user has queried with a name
your query has returned CCN1C=C[N+](=C1)C
long anion name
database found
user has queried with a name
your query has returned CCOS(=O)(=O)[O-]

database found
user has queried with a name
your query has returned CCN1C=C[N+](=C1)C
long anion name
database found
user has queried with a name
your query has returned CCOS(=O)(=O)[O-]

database found
user has queried with a name
your query has returned CCN1C=C[N+](=C1)C
long anion name
database found
user has queried with a name
your query has returned CCOS(=O)(=O)[O-]

database found
user has queried with a name
your query has returned CCN1C=C[N+](=C1)C
long anion name
database found
user has queried with a name
your query has returned CCOS(=O)(=O)[O-]

database found
user has queried with a name
your query has returned CCN1C=C[N+](=C1)C
long anion name
database found
user has queried with a name
your query has returned CCOS(=O

database found
user has queried with a name
your query has returned CCN1C=C[N+](=C1)C
long anion name
database found
user has queried with a name
your query has returned CCOS(=O)(=O)[O-]

database found
user has queried with a name
your query has returned CCN1C=C[N+](=C1)C
long anion name
database found
user has queried with a name
your query has returned CCOS(=O)(=O)[O-]

database found
user has queried with a name
your query has returned CCN1C=C[N+](=C1)C
long anion name
database found
user has queried with a name
your query has returned CCOS(=O)(=O)[O-]

database found
user has queried with a name
your query has returned CCN1C=C[N+](=C1)C
long anion name
database found
user has queried with a name
your query has returned CCOS(=O)(=O)[O-]

database found
user has queried with a name
your query has returned CCN1C=C[N+](=C1)C
long anion name
database found
user has queried with a name
your query has returned CCOS(=O)(=O)[O-]

database found
user has queried with a name
your query has r

long anion name
database found
user has queried with a name
your query has returned CCOS(=O)(=O)[O-]

database found
user has queried with a name
your query has returned CCN1C=C[N+](=C1)C
long anion name
database found
user has queried with a name
your query has returned CCOS(=O)(=O)[O-]

database found
user has queried with a name
your query has returned CCN1C=C[N+](=C1)C
long anion name
database found
user has queried with a name
your query has returned CCOS(=O)(=O)[O-]

database found
user has queried with a name
your query has returned CCN1C=C[N+](=C1)C
long anion name
database found
user has queried with a name
your query has returned CCOS(=O)(=O)[O-]

database found
user has queried with a name
your query has returned CCN1C=C[N+](=C1)C
long anion name
database found
user has queried with a name
your query has returned CCOS(=O)(=O)[O-]

database found
user has queried with a name
your query has returned CCN1C=C[N+](=C1)C
long anion name
database found
user has queried with a name


user has queried with a name
your query has returned CCOS(=O)(=O)[O-]

database found
user has queried with a name
your query has returned CCN1C=C[N+](=C1)C
long anion name
database found
user has queried with a name
your query has returned CCOS(=O)(=O)[O-]

database found
user has queried with a name
your query has returned CCN1C=C[N+](=C1)C
long anion name
database found
user has queried with a name
your query has returned CCOS(=O)(=O)[O-]

database found
user has queried with a name
your query has returned CCN1C=C[N+](=C1)C
long anion name
database found
user has queried with a name
your query has returned CCOS(=O)(=O)[O-]

database found
user has queried with a name
your query has returned CCN1C=C[N+](=C1)C
long anion name
database found
user has queried with a name
your query has returned CCOS(=O)(=O)[O-]

database found
user has queried with a name
your query has returned CCN1C=C[N+](=C1)C
long anion name
database found
user has queried with a name
your query has returned CCOS(=O

,Unnamed: 0,Unnamed: 0.1,index,Unnamed: 0.1.1,"Molar volume, m<SUP>3</SUP>/mol","Pressure, kPa","Specific density, kg/m<SUP>3</SUP>","Specific volume, m<SUP>3</SUP>/kg","Temperature, K",salt_name,salt_SMILES,cation_name,anion_name,cation_SMILES,anion_SMILES
0,1,1,1,0,NaN,101.325,1518.90,NaN,298.15,1-ethyl-3-methylimidazolium bis[(trifluorometh...,"CCN1C=C[N+](=C1)C,C(F)(F)(F)S(=O)(=O)[N-]S(=O)...",1-ethyl-3-methylimidazolium,bis[(trifluoromethyl)sulfonyl]imide,CCN1C=C[N+](=C1)C,C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F
1,11,11,11,0,NaN,101.325,1296.00,NaN,293.10,1-ethyl-3-methylimidazolium tetrafluoroborate,"CCN1C=C[N+](=C1)C,[B-](F)(F)(F)F",1-ethyl-3-methylimidazolium,tetrafluoroborate,CCN1C=C[N+](=C1)C,[B-](F)(F)(F)F
2,61,61,61,0,NaN,101.300,1279.00,NaN,298.15,1-ethyl-3-methylimidazolium tetrafluoroborate,"CCN1C=C[N+](=C1)C,[B-](F)(F)(F)F",1-ethyl-3-methylimidazolium,tetrafluoroborate,CCN1C=C[N+](=C1)C,[B-](F)(F)(F)F
3,62,62,62,0,NaN,101.300,1228.00,NaN,298.15,1-ethyl-3-methylimidazolium ethyl sulfate,"CCN1C=C[N+](=C1)C,CCOS(=O)(=O)[O-]\n",1-ethyl-3-methylimidazolium,ethyl sulfate,CCN1C=C[N+](=C1)C,CCOS(=O)(=O)[O-]\n
4,66,66,66,0,NaN,101.325,1383.60,NaN,298.15,1-ethyl-3-methylimidazolium trifluoromethanesu...,"CCN1C=C[N+](=C1)C,C(F)(F)(F)S(=O)(=O)[O-]",1-ethyl-3-methylimidazolium,trifluoromethanesulfonate,CCN1C=C[N+](=C1)C,C(F)(F)(F)S(=O)(=O)[O-]
5,67,67,67,0,NaN,101.325,1383.60,NaN,298.15,1-ethyl-3-methylimidazolium trifluoromethanesu...,"CCN1C=C[N+](=C1)C,C(F)(F)(F)S(=O)(=O)[O-]",1-ethyl-3-methylimidazolium,trifluoromethanesulfonate,CCN1C=C[N+](=C1)C,C(F)(F)(F)S(=O)(=O)[O-]
6,68,68,68,0,NaN,101.325,1252.60,NaN,333.10,1-ethyl-3-methylimidazolium tetrafluoroborate,"CCN1C=C[N+](=C1)C,[B-](F)(F)(F)F",1-ethyl-3-methylimidazolium,tetrafluoroborate,CCN1C=C[N+](=C1)C,[B-](F)(F)(F)F
7,97,97,97,0,NaN,101.000,1280.40,NaN,298.15,1-ethyl-3-methylimidazolium tetrafluoroborate,"CCN1C=C[N+](=C1)C,[B-](F)(F)(F)F",1-ethyl-3-methylimidazolium,tetrafluoroborate,CCN1C=C[N+](=C1)C,[B-](F)(F)(F)F
8,98,98,98,0,NaN,101.000,1283.70,NaN,298.15,1-ethyl-3-methylimidazolium tetrafluoroborate,"CCN1C=C[N+](=C1)C,[B-](F)(F)(F)F",1-ethyl-3-methylimidazolium,tetrafluoroborate,CCN1C=C[N+](=C1)C,[B-](F)(F)(F)F
9,124,124,124,0,NaN,101.325,1279.00,NaN,297.00,1-ethyl-3-methylimidazolium tetrafluoroborate,"CCN1C=C[N+](=C1)C,[B-](F)(F)(F)F",1-ethyl-3-methylimidazolium,tetrafluoroborate,CCN1C=C[N+](=C1)C,[B-](F)(F)(F)F


In [29]:
pd.DataFrame.to_csv(salts, path_or_buf='../salty/data/salts_with_smiles.csv', index=False)

In [37]:
data=pd.read_csv('../salty/data/salts_with_smiles.csv',delimiter=',')
print(data.columns)
n = data.shape[0]
f = open('../salty/data/Deslist','r')
Deslist = []
for line in f:
    Deslist.append(line.strip('\n\t'))
calc = Calculator(Deslist)
D = len(Deslist)
d = len(Deslist)*2+3
print(n,d)

Index([u'Molar volume, m<SUP>3</SUP>/mol', u'Pressure, kPa',
       u'Specific density, kg/m<SUP>3</SUP>',
       u'Specific volume, m<SUP>3</SUP>/kg', u'Temperature, K', u'salt_name',
       u'salt_SMILES', u'cation_name', u'anion_name', u'cation_SMILES',
       u'anion_SMILES'],
      dtype='object')
3702 191


In [38]:
X = np.zeros((n,d))
X[:,-3] = data['Temperature, K']
X[:,-2] = data['Pressure, kPa']
for i in range(n):
    cation = Chem.MolFromSmiles(data['cation_SMILES'][i])
    anion  = Chem.MolFromSmiles(data['anion_SMILES'][i])
    X[i][:D]    = calc.CalcDescriptors(cation)
    X[i][D:2*D] = calc.CalcDescriptors(anion)

X[:,-1] = data['Specific density, kg/m<SUP>3</SUP>']
print(X.shape)

np.savetxt("../salty/data/salts_with_descriptors",X)

(3702, 191)
